In [5]:
# coding: utf-8

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import sqlite3
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.sql import select
from datetime import datetime

In [2]:
sys.path.append(r"../src/collectcube")
from db import *

### Check that pixel table already exists

In [3]:
sql_db_path = 'sqlite:///../data/db_samp/landcover.db'
engine = create_engine(sql_db_path, echo=False)
with engine.connect() as conn:
    df = pd.read_sql_table('pixels', conn)
df.head()

,PID,Center,cent_lat,cent_long,cent_X,cent_Y,ransamp,checked,PID0,PID1
0,0000001_0,1,-22.692581,-55.655989,3168760.0,-2877060.0,1,0,1,0
1,0000002_0,1,-26.212240,-55.232312,3167140.0,-3308080.0,1,0,2,0
2,0000003_0,1,-26.454174,-55.398933,3148970.0,-3337470.0,1,0,3,0
3,0000004_0,1,-25.230882,-56.398586,3072420.0,-3188540.0,1,0,4,0
4,0000005_0,1,-24.552260,-56.964313,3028280.0,-3105580.0,1,0,5,0


## Make sql lc table from existing lut.csv

In [4]:
local_db_path = '../data/db_samp/landcover.db'
lc_lut = Path("../data/Class_LUT.csv")
make_LC_table_from_lut(lc_lut, local_db_path, treat_existing='fail')

### check if it worked:

In [5]:
with engine.connect() as con:
    df = pd.read_sql_table('LC', con)
df.tail()

,LC_UNQ,USE_NAME,LC5_name,LC5,LC25
39,68,Trees-Forest_palm,Trees,70,68
40,70,Trees-Wetland,Trees,70,70
41,80,Trees-Forest,Trees,70,80
42,98,N_NoCrop,unknown,99,98
43,99,unknown,unknown,99,99


### Add other tables to database using sqlalchemy to control foreign keys

In [6]:
def make_db(sql_db_path):
    engine = create_engine(sql_db_path, echo=False)
    make_main_support_tables(engine)
    populate_LC5_table(engine)
    
make_db(sql_db_path)

added LC5 and PixVar tables to db


### Check that tables are made and populated

In [7]:
with engine.connect() as conn:
    lc5_table = Table('LC5', MetaData(), autoload_with=engine)
    rp = conn.execute(lc5_table.select())
results = rp.fetchall() 
print(results)

[(0, ' ------'), (10, 'NoVeg'), (20, 'LowVeg'), (40, 'MedVeg'), (50, 'HighVeg'), (70, 'Trees'), (99, 'Unknown')]


In [8]:
with engine.connect() as con:
    df = pd.read_sql_table('LC5', con)
df.head()

,LC5id,LC5type
0,0,------
1,10,NoVeg
2,20,LowVeg
3,40,MedVeg
4,50,HighVeg


In [9]:
with engine.connect() as con:
    df = pd.read_sql_table('PixelVerification', con)
df.head()

,recID,PID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,...,DEAD,MEDVEG,TREEPLANT0,HIGHVEG,TREEPLANT,FOREST,Age,Stability,State,Notes


In [10]:
## first record:
testDate = datetime(2000,1,1)
with engine.connect() as conn:
    pixvar = Table('PixelVerification', MetaData(), autoload_with=engine)
    query = sa.insert(pixvar).values(PID='0_0', PID0=0, PID1=0, imgDate=testDate,LC5=0,LC=0,HOMONBHD9=8) 
    go = conn.execute(query)
    conn.commit()
print(go.inserted_primary_key)

(1,)
